In [1]:
import os
import sys
from dotenv import load_dotenv

load_dotenv()
sys.path.append(os.environ.get('PATH_CUSTOM_MODULES'))

import modeling

### Prepare all basic variable

In [2]:
path_source = os.environ.get('PATH_DATASET_DESTINATION')
path_pretrained = os.environ.get('PATH_PRETRAINED')
path_model_storage = os.environ.get('PATH_MODEL')
scenario_names = ['scenario_1', 'scenario_2', 'scenario_3', 'scenario_4', 'scenario_5']
dataset_names = ['rimone', 'g1020', 'refuge', 'papila']
fold_names = ['fold_1', 'fold_2', 'fold_3', 'fold_4', 'fold_5']
image_size = {'rimone': (300,300),
            'g1020': (240,300),
            'refuge': (300,300),
            'papila': (200,300)}
col_mode = {'scenario_1': 'rgb',
            'scenario_2': 'rgb',
            'scenario_3': 'rgb',
            'scenario_4': 'grayscale',
            'scenario_5': 'grayscale'}
pre_trained_models = ['mobilenet_v2', 'mobilenet_v3small', 'mobilenet_v3large',
                'efficientnet_v2s', 'efficientnet_v2m', 'efficientnet_v2l']

### Prepare the path source

In [3]:
path_dataset_src = {}

for scenario in scenario_names:
    for dataset in dataset_names:
        for fold in fold_names:
            if scenario == 'scenario_1':
                    train = 'train'
            else:
                train = 'train_augmented'

            for data_type in [train, 'val', 'test']:
                path_dataset_src[f'{scenario}_'
                                + f'{dataset}_'
                                + f'{fold}_'
                                + f'{data_type}'] = os.path.join(path_source,
                                                                scenario,
                                                                dataset,
                                                                fold,
                                                                data_type)
del scenario, dataset, fold, data_type, train

### Prepare the image data generator

In [7]:
img_gen = modeling.datagen(scenario_names=scenario_names,
                            dataset_names=dataset_names,
                            fold_names=fold_names,
                            path_dataset_src=path_dataset_src,
                            image_size=image_size,
                            col_mode=col_mode)

Creating image data generator for scenario_1 rimone fold_1 train
Found 339 images belonging to 2 classes.
Creating image data generator for scenario_1 rimone fold_1 val
Found 49 images belonging to 2 classes.
Creating image data generator for scenario_1 rimone fold_1 test
Found 97 images belonging to 2 classes.
Creating image data generator for scenario_1 rimone fold_2 train
Found 339 images belonging to 2 classes.
Creating image data generator for scenario_1 rimone fold_2 val
Found 49 images belonging to 2 classes.
Creating image data generator for scenario_1 rimone fold_2 test
Found 97 images belonging to 2 classes.
Creating image data generator for scenario_1 rimone fold_3 train
Found 339 images belonging to 2 classes.
Creating image data generator for scenario_1 rimone fold_3 val
Found 49 images belonging to 2 classes.
Creating image data generator for scenario_1 rimone fold_3 test
Found 97 images belonging to 2 classes.
Creating image data generator for scenario_1 rimone fold_4 tr

### Create model

In [5]:
model = {}
for pretrain in pre_trained_models:
    for color_mode in set(col_mode.values()):
        if color_mode == 'rgb':
            col_channels = 3
        elif color_mode == 'grayscale':
            col_channels = 1
        
        for img_shape in set(image_size.values()):
            print(type(img_shape + (col_channels,)),
                img_shape + (col_channels,))
            model[f'{pretrain}_'
                + f'{color_mode}_'
                + f'{img_shape}'] = modeling.model_base(model_name=pretrain,
                                                        img_shape=img_shape+(col_channels,),
                                                        path_model_src=path_pretrained)
            break
        break
    break

<class 'tuple'> (240, 300, 1)


ValueError: The input must have 3 channels; Received `input_shape=(240, 300, 1)`

### testing

In [6]:
for model in [
    # 'mobilenet_v2', 'mobilenet_v3small', 'mobilenet_v3large',
                'efficientnet_v2s', 'efficientnet_v2m', 'efficientnet_v2l']:
    model = modeling.model_base(model_name=model,
                                img_shape=(image_size['g1020'][0],
                                            image_size['g1020'][1],
                                            1),
                                path_model_src=path_pretrained)
    break

ValueError: Cannot assign value to variable ' stem_conv/kernel:0': Shape mismatch.The variable shape (3, 3, 1, 24), and the assigned value shape (24, 3, 3, 3) are incompatible.

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

In [ ]:
model.fit(img_gen['scenario_1_rimone_fold_1_train'],
            epochs=15,
            validation_data=img_gen['scenario_1_rimone_fold_1_val'])